# Using the Equities India (NSE) data-set

In [1]:
from sqlalchemy import func, and_, or_
from plutoPy.model import RoutingSession, EquitiesIndiaNse
from datetime import date, datetime

### fetch the earliest 10 listed equity

In [2]:
results = (RoutingSession.session.query(EquitiesIndiaNse.Tickers)
           .order_by(EquitiesIndiaNse.Tickers.DATE_LISTING)
           .limit(10)
           .all())

for instance in results:
    print(instance)

AARTIIND(BE): Aarti Industries Limited, 1995-Feb-08
ABB(EQ): ABB India Limited, 1995-Feb-08
AMJLAND(EQ): Amj Land Holdings Limited, 1995-Feb-08
ARVIND(EQ): Arvind Limited, 1995-Feb-08
BALRAMCHIN(EQ): Balrampur Chini Mills Limited, 1995-Feb-08
BLUEBLENDS(BE): Blue Blends (I) Limited, 1995-Feb-08
CESC(EQ): CESC Limited, 1995-Feb-08
CIPLA(EQ): Cipla Limited, 1995-Feb-08
EIHOTEL(EQ): EIH Limited, 1995-Feb-08
FEDERALBNK(EQ): The Federal Bank  Limited, 1995-Feb-08


/usr/local/lib/python3.6/dist-packages/sqlalchemy/dialects/mssql/base.py:2307: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42S02', "[42S02] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Invalid object name 'sys.dm_pdw_nodes_exec_sessions'.")
  "tried views: %s; final error was: %s" % (views, last_error)


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)